In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from pathlib import Path
import nltk
import math

In [12]:
doc_dict = {}
counter = 0

for f in Path('/Users/robert/Documents/Uni/DataScience/Übung5').glob('*.txt'):
    doc_dict.update({f.name:f.read_text('utf-8')})
    counter+=1

In [13]:
for f_name in doc_dict:
    doc_dict[f_name] = nltk.word_tokenize(doc_dict[f_name], language='german')

## Calculate Word Frequencies per Document

In [14]:
bags_of_words = {}
compl_bag_of_words = set()

for f_name in doc_dict:
    bags_of_words.update({f_name: {}})
    for word in doc_dict[f_name]:
        if word.lower() in bags_of_words[f_name]:
            bags_of_words[f_name][word.lower()] = bags_of_words[f_name][word.lower()] + 1
        else:
            bags_of_words[f_name].update({word.lower(): 1})
        compl_bag_of_words.add(word)

## Now the cosine similarity (greatest will be clustered together in the next step)

In [15]:
#length of vectors
len_doc_vector = {}
for f_name in bags_of_words:
    length = 0
    for word in bags_of_words[f_name]:
        length += (bags_of_words[f_name][word])*(bags_of_words[f_name][word])
    len_doc_vector.update({f_name: math.sqrt(length)})

#dot product
docs = list(bags_of_words.keys())
dot_ps = {(docs[0], docs[1]): 0, (docs[0],docs[2]):0, (docs[1],docs[2]): 0}

sims = {(0,1): {},(0,2): {},(1,2): {}}

for word in compl_bag_of_words:
    if word in bags_of_words[docs[0]] and word in bags_of_words[docs[1]]:
        dot_ps[(docs[0],docs[1])] += bags_of_words[docs[0]][word]*bags_of_words[docs[1]][word]

    if word in bags_of_words[docs[0]] and word in bags_of_words[docs[2]]:
        dot_ps[(docs[0],docs[2])] += bags_of_words[docs[0]][word]*bags_of_words[docs[2]][word]

    if word in bags_of_words[docs[1]] and word in bags_of_words[docs[2]]:
        dot_ps[(docs[1],docs[2])] += bags_of_words[docs[1]][word]*bags_of_words[docs[2]][word]

cos_sims = dot_ps.copy()

for combination in cos_sims:
    cos_sims[combination]  = cos_sims[combination] / (len_doc_vector[combination[0]] * len_doc_vector[combination[1]])

In [16]:
cos_sims

{('erlkonig.txt', 'kollegah-universalgenie1strophe.txt'): 0.5256618065582226,
 ('erlkonig.txt', 'erlkoniganalyse.txt'): 0.5425230687931862,
 ('kollegah-universalgenie1strophe.txt',
  'erlkoniganalyse.txt'): 0.5319882843361549}

## Euklidian distance

In [17]:
eucl_dist = {(docs[0], docs[1]): 0, (docs[0],docs[2]):0, (docs[1],docs[2]): 0}

for combination in eucl_dist:
    for word in compl_bag_of_words:
        if word in bags_of_words[combination[0]] and word in bags_of_words[combination[1]]:
            eucl_dist[combination] += math.pow(bags_of_words[combination[0]][word] - bags_of_words[combination[1]][word], 2)
        elif word in bags_of_words[combination[0]]:
            eucl_dist[combination] += math.pow(bags_of_words[combination[0]][word], 2)
        elif word in bags_of_words[combination[1]]:
            eucl_dist[combination] += math.pow(bags_of_words[combination[1]][word], 2)

    eucl_dist[combination] = math.sqrt(eucl_dist[combination])

In [18]:
eucl_dist

{('erlkonig.txt', 'kollegah-universalgenie1strophe.txt'): 78.58116822750856,
 ('erlkonig.txt', 'erlkoniganalyse.txt'): 106.5926826756884,
 ('kollegah-universalgenie1strophe.txt',
  'erlkoniganalyse.txt'): 105.75916035975324}

## Begründung:

An der euklidischen Distanz ist erkennbar, dass,
wenn nun aus den elementaren Clustern (den Datenpunkten) ein zusammengefügtes Cluster nach Complete Link erstellt wird,
zunächst ein Cluster aus erlkonig und kollegahs Universalgenie entsteht, während dies bei Cosine Similarity und Single Link stattdessen
universalgenie (Strophe 1)  und erlkoniganalyse wären.